RUN CODE FROM HERE FOR REAL TIME PROCESSING!!

In [1]:
import os
import socket
import math
import time
import sys
from time import sleep
from threading import Timer
import matplotlib.pyplot as plt
import numpy as np
import queue
import threading
import open3d as o3d
from sklearn.cluster import DBSCAN
import scipy

In [2]:
# Socket Testing
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.bind(('', 2368))

time_start = time.perf_counter()

for i in range(1510):
    recvData = s.recv(1248)
    
print("Time Taken :", time.perf_counter() - time_start)

s.close()

Time Taken : 1.0016829579981277


In [3]:
# 传感器信息
marker = b'\xffee'
n_azimuth_bytes = 2
n_channels = 32
n_data_block_bytes = 100
n_channel_data_bytes = 3
n_data_blocks = 12

# This following data
# The elevation angle
pitch_deg = [-25.0, -1.0, -1.667, -15.639, -11.31, 0.0, -0.667, -8.843,
             -7.254, 0.333, -0.333, -6.148, -5.333, 1.333, 0.667, -4.0,
             -4.667, 1.667, 1.0, -3.667, -3.333, 3.333, 2.333, -2.667,
             -3.0, 7.0, 4.667, 2.333, -2.0, 15.0, 10.333, -1.333]
# The azimuth offset
delta_deg = [1.4, -4.2, 1.4, -1.4, 1.4, -1.4, 4.2, -1.4,
             1.4, -4.2, 1.4, -1.4, 4.2, -1.4, 4.2, -1.4,
             1.4, -4.2, 1.4, -4.2, 4.2, -1.4, 1.4, -1.4,
             1.4, -1.4, 1.4, -4.2, 4.2, -1.4, 1.4, -1.4]
deg2rad = math.pi / 180.0
rad2deg = 180.0 / math.pi
dist_meas = 4.0
s_name = str(dist_meas) + '.csv'  # 存储的文件名

def mlab_plot3d(x, y, z):
    mlab.points3d(x, y, z)
    mlab.show()


# 输入三字节数据，返回距离值
def get_distance(data_per_channel):
    distance = (int(data_per_channel[1]) * 256 + int(data_per_channel[0])) * 4.0 / 1000.0
    return distance


# 输入channel值、距离值、航向角度，返回(x,y,z)
def get_xyz(channel, dist, azimuth):
    """
    x = dist * math.cos(pitch_deg[channel] * deg2rad) * math.sin((azimuth + delta_deg[channel]) * deg2rad)
    y = dist * math.cos(pitch_deg[channel] * deg2rad) * math.cos((azimuth + delta_deg[channel]) * deg2rad)
    z = dist * math.sin(pitch_deg[channel] * deg2rad)
    """
    x = dist * math.cos(pitch_deg[channel] * deg2rad) * math.sin(azimuth * deg2rad)
    y = dist * math.cos(pitch_deg[channel] * deg2rad) * math.cos(azimuth * deg2rad)
    z = dist * math.sin(pitch_deg[channel] * deg2rad)
    return x, y, z


# 输入两字节数据，返回角度值
def get_azimuth(azimuth_data):
    angle = (int(azimuth_data[1]) * 256 + int(azimuth_data[0])) / 100.0
    return angle


def get_precision_azimuth(data_packet):  # 传引用
    i = 0
    azimuth_gap = 0.0
    for i in range(12):
        if i < 11:
            if data_packet[i + 1]['Azimuth'] < data_packet[i + 1]['Azimuth']:
                data_packet[i + 1]['Azimuth'] += 360
            azimuth_gap = data_packet[i + 1]['Azimuth'] - data_packet[i]['Azimuth']
        j = 0
        while j < 32:
            data_packet[i]['channel %d' % (j)][0] = data_packet[i]['Azimuth'] + azimuth_gap * j * 2.304 / 55.296 + \
                                                    delta_deg[j]
            data_packet[i]['channel %d' % (j + 1)][0] = data_packet[i]['Azimuth'] + azimuth_gap * j * 2.304 / 55.296 + \
                                                        delta_deg[j + 1]
            if data_packet[i]['channel %d' % (j)][0] > 360:
                data_packet[i]['channel %d' % (j)][0] -= 360

            elif data_packet[i]['channel %d' % (j)][0] < 0:
                data_packet[i]['channel %d' % (j)][0] += 360

            if data_packet[i]['channel %d' % (j + 1)][0] > 360:
                data_packet[i]['channel %d' % (j + 1)][0] -= 360

            elif data_packet[i]['channel %d' % (j + 1)][0] < 0:
                data_packet[i]['channel %d' % (j + 1)][0] += 360

            j += 2


# 每个data_packet包含12个data_block, data_block形式如下，每个channel包含azimuth和distance：
"""
data_block = {'Azimuth': .0,
              'channel 0': [.0, .0], 'channel 1': [.0, .0], 'channel 2': [.0, .0], ' channel 3': [.0, .0], 
              'channel 4': [.0, .0], 'channel 5': [.0, .0], 'channel 6': [.0, .0], 'channel 7': [.0, .0], 
              'channel 8': [.0, .0], 'channel 9': [.0, .0], 'channel 10': [.0, .0], 'channel 11': [.0, .0], 
              'channel 12': [.0, .0], 'channel 13': [.0, .0], 'channel 14': [.0, .0], 'channel 15': [.0, .0], 
              'channel 16': [.0, .0], 'channel 17': [.0, .0], 'channel 18': [.0, .0], 'channel 19': [.0, .0], 
              'channel 20': [.0, .0], 'channel 21': [.0, .0], 'channel 22': [.0, .0], 'channel 23': [.0, .0], 
              'channel 24': [.0, .0], 'channel 25': [.0, .0], 'channel 26': [.0, .0], 'channel 27': [.0, .0], 
              'channel 28': [.0, .0], 'channel 29': [.0, .0], 'channel 30': [.0, .0], 'channel 31': [.0, .0]}
"""


# 返回data packet
def getinfo(recv_Data):
    data_packet = []
    for i in range(n_data_blocks):
        data_block = dict()
        block_azimuth = get_azimuth(
            recv_Data[i * n_data_block_bytes + 2: i * n_data_block_bytes + 4])  # ffee标志位后的两字节为azimuth
        data_block.update({'Azimuth': block_azimuth})
        for j in range(n_channels):
            s_t = 'channel ' + str(j)
            dist = get_distance(recv_Data[
                                i * n_data_block_bytes + 4 + j * n_channel_data_bytes: i * n_data_block_bytes + 7 + j * n_channel_data_bytes])
            data_block.update({s_t: [block_azimuth, dist]})  # 此处尚未计及精确的azimuth
        data_packet.append(data_block)
    get_precision_azimuth(data_packet)
    return data_packet

# Functions

# Rotating the point clouds to be consistent with the video

def rotation_matrix(theta1, theta2, theta3, order='xyz'):
    c1 = np.cos(theta1 * np.pi / 180)
    s1 = np.sin(theta1 * np.pi / 180)
    c2 = np.cos(theta2 * np.pi / 180)
    s2 = np.sin(theta2 * np.pi / 180)
    c3 = np.cos(theta3 * np.pi / 180)
    s3 = np.sin(theta3 * np.pi / 180)
    matrix = np.array([[c2 * c3, -c2 * s3, s2],
                       [c1 * s3 + c3 * s1 * s2, c1 * c3 - s1 * s2 * s3, -c2 * s1],
                       [s1 * s3 - c1 * c3 * s2, c3 * s1 + c1 * s2 * s3, c1 * c2]])
    return matrix

# Converting to spherical co-ordinates 
def cart2sphA(xyz):
    ptsnew = np.zeros(xyz.shape)
    xy = xyz[:,0]**2 + xyz[:,1]**2
    ptsnew[:,0] = np.sqrt(xy + xyz[:,2]**2)
#     ptsnew[:,1] = np.arctan2(np.sqrt(xy), xyz[:,2]) # for elevation angle defined from Z-axis down
    ptsnew[:,1] = np.arctan2(xyz[:,2], np.sqrt(xy)) # for elevation angle defined from XY-plane up
    ptsnew[:,2] = np.arctan2(xyz[:,1], xyz[:,0])
    return ptsnew

# Cropping point cloud data.

def cropPtCloud(ptCloud):
    
    xLim = [-150, 150]
    yLim = [-150, 150]
    zLim = [-10, 10]
        
    insideX = np.logical_and(ptCloud[:,0] < xLim[1], ptCloud[:,0] > xLim[0])
    insideY = np.logical_and(ptCloud[:,1] < yLim[1], ptCloud[:,1] > yLim[0])
    insideZ = np.logical_and(ptCloud[:,2] < zLim[1], ptCloud[:,2] > zLim[0])
    
    validIndices = np.logical_and(np.logical_and(insideX, insideY), insideZ)
            
    return ptCloud[validIndices,:]

In [15]:
class PacketCapture:

    def __init__(self):
        
        # Connecting to Port 2368
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.socket.bind(('', 2368))
        
        self.done = False  # Termination Flag
        self.frame = []
        
        # Generating Parallel Thread for the Capture.
        self.q = queue.Queue()
        t = threading.Thread(target=self._reader)
        t.daemon = True
        t.start()
        


    # read frames as soon as they are available, keeping only most recent 151
    def _reader(self):
        
        while not self.done:
            self.frame.append(self.socket.recv(1248))            
            
            if len(self.frame) > 151: # ~151 Data Packets in one complete rotation of the LiDAR with 600 RPM
                self.frame.pop()
                
            if not self.q.empty():
                try:
                    self.q.get_nowait()   # discard previous (unprocessed) frame
                except queue.Empty:
                    pass
            self.q.put(self.frame)

    def read(self):
        return self.q.get()
    
    def close(self):
        self.done = True
        self.socket.close()


In [16]:
capture = PacketCapture()

# This is processing and capturing one frame of LiDAR Data. We can put this code in a while loop to convert it into continuous time.
# First we will make sure that this runs properly then do Real time processing.
time_start = time.perf_counter()

range_thrld_matrix = scipy.io.loadmat('range_thrld_matrix.mat')
range_thrld_matrix = range_thrld_matrix['range_thrld_matrix']
beams_elevation = scipy.io.loadmat('beams_elevation_vlp32c.mat')
beams_elevation = beams_elevation['beams_elevation_vlp32c']

total_grid = 1800;
azimuth_unit = 360/total_grid;

pt_cloud = np.zeros((10000000, 3))
counter = 0
packet_count = 0

# Reading and processing the capture data frame
recvData = capture.read()
print(len(recvData))
for n in range(151):
    d_p = getinfo(recvData[n])    
    for i in range(12):      # No. of firing sequences in one Data Packet 
        for channel_n in range(32):     # Total no. of channels in VLP-32C LiDAR 
            pt_cloud[counter,:] = get_xyz(pitch_deg[channel_n], d_p[i]['channel %d' % (channel_n)][1], d_p[i]['channel %d' % (channel_n)][0])
            counter += 1

# Filtering
'''
1. Drop all the coordinates with z less than a threshold.
'''

# Cropping the point cloud data and obtaining .
cloud_array = cropPtCloud(pt_cloud[:counter]);

# Convert Cartesian to Spherical.
cloud_sph_arr = cart2sphA(cloud_array[:,0:3])

# Converting Azimuth from radians to degrees [0, 360]
degree = cloud_sph_arr[:,2]
degree = degree*180/3.14; # rad to deg
degree[degree<0] = degree[degree<0] + 360

# Assigning grids based on azimuth angle
grids_idx = (np.floor(degree/azimuth_unit) + 1)%total_grid
grids_idx[grids_idx == 0] = total_grid - 1  # deal with 1799 + 1 grid
grids_idx[grids_idx == None] = 0 # LiDAR point NaN is set as 0
grids_idx = grids_idx.astype(int)

back_idxes = np.zeros(len(cloud_sph_arr))
back_idxes[grids_idx == 0] = 1

elevation_matrix = abs(np.zeros((len(beams_elevation),len(cloud_sph_arr))) + cloud_sph_arr[:,1] - beams_elevation)
channel_num = np.argmin(elevation_matrix, axis = 0)

mask = cloud_sph_arr[:,0] >= range_thrld_matrix[channel_num, grids_idx]
back_idxes[mask] = 1

filtered_point_cloud = cloud_array[back_idxes == 0, 0:3]

# Rotating the point clouds to be consistent with the video
tmp = rotation_matrix(0, 0, -43).dot(filtered_point_cloud.T).T
tmp = rotation_matrix(25, -5, 0).dot(tmp.T).T
filtered_point_cloud_1 = rotation_matrix(0, 0, 0).dot(tmp.T).T

# Sklearn DBSCAN Implementation
X = filtered_point_cloud_1[:,0:2]
start_time  = time.time()
db = DBSCAN(eps=1, min_samples=17, algorithm='ball_tree', n_jobs=1).fit(X)
end_time = time.time()
timings.append(end_time-start_time)
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)


# Storing a rectangle for each cluster for later plotting on image
rectangle_list = []

for j in range(n_clusters_):
    class_member_mask = labels == j
    xy = X[class_member_mask, :]
    rectangle = [(int(125 + 20.5*np.min(xy[:, 0])), int(735 - 20*np.max(xy[:, 1]))), (int(125 + 20.5*np.max(xy[:, 0])), int(735 - 20*np.min(xy[:, 1])))]
    rectangle_list.append(rectangle)
clustered_point_cloud = (rectangle_list)    
    
print("Time Taken :", time.perf_counter() - time_start)
print("Data Points :", counter)

capture.close()

151
Time Taken : 0.17844720799985225
Data Points : 57984


In [9]:
# Open3D Visualization
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pt_cloud)
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
